In [2]:
import face_recognition as fr
import pandas as pd
import cv2
import numpy as np

In [ ]:
df0 = pd.DataFrame({'name':['shubham','yash' ],'enc':['4','5']})
df0

,name,enc
0,shubham,4
1,yash,5


In [ ]:
df1 = pd.DataFrame({'name':['kashish','yashu' ],'enc':['4','5']})
df1

,name,enc
0,kashish,4
1,yashu,5


In [ ]:
pd.concat([df0,df1])

,name,enc
0,shubham,4
1,yash,5
0,kashish,4
1,yashu,5


In [4]:
#face registration
#####################
fname = 'features.csv'

try:
    df=pd.read_csv(fname)
except:
    df = pd.DataFrame({'name':[],'enc':[]})

fd = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)
vid = cv2.VideoCapture(0)
counter = 0
names =[]
feats = []  
name = input('Enter your name:')
while True:
    ack, img = vid.read()
    if ack:
        faces = fd.detectMultiScale(img, 1.2,10)
        if len(faces) == 1:
            x,y,w,h = faces[0]
            face_img = img[y:y+h,x:x+w,:].copy()
            face_enc = fr.face_encodings(face_img)
            if len(face_enc) == 1:
                counter += 1
                print(counter)
                names += [name]
                feats += [face_enc[0].tolist()]
                #f= pd.DataFrame({'name':[name], 'enc':face_enc})
                #df = pd.concat([df,f],axis=0, ignore_index=True
                
        #...
            if counter == 20:
                f= pd.DataFrame({'name':names, 'enc':feats})
                df = pd.concat([df,f],axis=0, ignore_index=True)
                df.to_csv(fname)
                print('saved')
                break 
        cv2.imshow('preview', img) #depends on requirement
        key = cv2.waitKey(1)
        if key == ord('x'):
            break
cv2.destroyAllWindows()
vid.release()

1
2
3
4


In [5]:
#Recognition

try:
    df=pd.read_csv(fname)
except:
    print ("Face database not found. Halt")
else:
    
    fd = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    vid = cv2.VideoCapture(0)
    
    while True:
        ack, img = vid.read()
        if ack:
            faces = fd.detectMultiScale(img, 1.2,10)
            if len(faces) == 1:
                x,y,w,h = faces[0]
                face_img = img[y:y+h,x:x+w,:].copy()
                face_enc = fr.face_encodings(face_img)
                if len(face_enc) == 1:
                    
                    feats_data = df['enc'].apply(lambda x:eval(x)).values.tolist()
                    matches = fr.compare_faces(face_enc, np.array(feats_data)) 

                    if True in matches:
                        match_ind = matches.index(True)
                        name = df['name'][match_ind]
                    else:
                        name = 'Unknown'

                    cv2.putText(
                        img, name, (150,150),
                        cv2.FONT_HERSHEY_DUPLEX,2,(0,0,255), thickness=5
                    )
                                        

            cv2.imshow('preview', img) #depends on requirement
            key = cv2.waitKey(1)
            if key == ord('x'):
                break
    cv2.destroyAllWindows()
    vid.release()